In [65]:
!git clone https://github.com/sakanaowo/PlantXViT


Cloning into 'PlantXViT'...
remote: Enumerating objects: 50338, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 50338 (delta 20), reused 57 (delta 12), pack-reused 50271 (from 1)
Receiving objects: 100% (50338/50338), 1.66 GiB | 60.80 MiB/s, done.
Resolving deltas: 100% (30405/30405), done.
Updating files: 100% (50030/50030), done.


In [66]:
%cd PlantXViT

/content/PlantXViT/PlantXViT


Preprocess from here


In [67]:
from utils.config_loader import load_config
config = load_config("configs/config.yaml")

In [68]:
print(config)

{'dataset': {'embrapa': {'image_size': [224, 224], 'data_dir': './data/raw/embrapa', 'label_encoder': './data/processed/embrapa_label_encoder.pkl'}, 'apple': {'image_size': [224, 224], 'data_dir': './data/raw/plant-pathology-2020-fgvc7/images', 'csv_path': './data/raw/plant-pathology-2020-fgvc7/train.csv', 'label_encoder': './data/processed/apple_label_encoder.pkl'}}, 'training': {'batch_size': 16, 'epochs': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'loss': 'categorical_crossentropy'}, 'model': {'patch_size': 5, 'transformer_blocks': 4, 'embedding_dim': 16, 'dropout_rate': 0.1}, 'output': {'embrapa': {'save_dir': './outputs/embrapa', 'model_path': './outputs/embrapa/models/plantxvit_best.pth', 'log_dir': './outputs/embrapa/logs', 'explain_dir': './outputs/embrapa/explain'}, 'apple': {'save_dir': './outputs/apple', 'model_path': './outputs/models/apple/plantxvit_best.pth', 'log_dir': './outputs/apple/logs', 'explain_dir': './outputs/apple/explain'}}}


In [69]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle


In [70]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

def process_split_with_check(split_dir, verbose=True):
    image_paths = []
    labels = []

    for label_name in sorted(os.listdir(split_dir)):
        label_path = os.path.join(split_dir, label_name)
        if not os.path.isdir(label_path):
            continue

        valid_files = [
            fname for fname in os.listdir(label_path)
            if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]

        # Bỏ qua class rỗng
        if len(valid_files) == 0:
            if verbose:
                print(f"⚠️ Bỏ qua class trống: {label_name}")
            continue

        for fname in valid_files:
            full_path = os.path.join(label_path, fname)
            if os.path.exists(full_path):
                image_paths.append(os.path.join(label_name, fname))
                labels.append(label_name)
            elif verbose:
                print(f"⚠️ Ảnh không tồn tại: {full_path}")

    return pd.DataFrame({'image_path': image_paths, 'label': labels})


In [71]:
# Đường dẫn
base_dir = "./data/raw/embrapa"

# Tạo dataframe cho từng tập
train_df = process_split_with_check(os.path.join(base_dir, "train"))
val_df = process_split_with_check(os.path.join(base_dir, "val"))
test_df = process_split_with_check(os.path.join(base_dir, "test"))

# Encode nhãn
label_encoder = LabelEncoder()
train_df['label_idx'] = label_encoder.fit_transform(train_df['label'])
val_df['label_idx'] = label_encoder.transform(val_df['label'])
test_df['label_idx'] = label_encoder.transform(test_df['label'])

# Lưu lại
output_dir = "./data/processed/embrapa"
os.makedirs(output_dir, exist_ok=True)
train_df.to_csv(os.path.join(output_dir, "train.csv"), index=False)
val_df.to_csv(os.path.join(output_dir, "val.csv"), index=False)
test_df.to_csv(os.path.join(output_dir, "test.csv"), index=False)

# Lưu label encoder
with open(os.path.join(output_dir, "label_encoder.pkl"), "wb") as f:
    pickle.dump(label_encoder, f)

print("✅ Đã tạo lại đầy đủ train/val/test CSV với kiểm tra ảnh.")


✅ Đã tạo lại đầy đủ train/val/test CSV với kiểm tra ảnh.


In [72]:
#checking

df = pd.read_csv("./data/processed/embrapa/train.csv")
print("✅ Số ảnh huấn luyện:", len(df))
print(df.head())


✅ Số ảnh huấn luyện: 29607
                                          image_path  \
0  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   
1  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   
2  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   
3  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   
4  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   

                                               label  label_idx  
0  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  
1  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  
2  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  
3  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  
4  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  


In [73]:
#checking
import os

def count_valid_images(folder):
    count = 0
    for f in os.listdir(folder):
        if f.lower().endswith(('.jpg', '.jpeg', '.png')):
            count += 1
    print(count)
    return count

base_dir = "./data/raw/embrapa/train"
class_stats = {}

for class_name in sorted(os.listdir(base_dir)):
    class_path = os.path.join(base_dir, class_name)
    if os.path.isdir(class_path):
        n = count_valid_images(class_path)
        class_stats[class_name] = n

# In các class có ít ảnh hoặc không có ảnh
for k, v in class_stats.items():
    if v < 5:
        print(f"⚠️ {k}: {v} ảnh hợp lệ")


95
105
1094
157
12
32
56
244
735
112
32
70
540
852
1352
41
26
648
1059
66
156
12
144
196
94
41
16
172
281
97
372
22
12
389
124
64
12
84
40
358
24
49
268
600
64
384
24
48
116
83
416
73
14
14
16
13
14
60
7
108
9
94
16
568
1950
2412
498
11
684
99
7
19
7
2425
12
1449
988
198
617
10
1475
825
798
52
240
13
236
424
512
32
381
85
52


create dataset class and dataloader from here

In [74]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd


class EmbrapaDataset(Dataset):
    def __init__(self, csv_path, root_dir, transform=None):
        """
        Args:
            csv_path (str): Đường dẫn đến CSV chứa image_path, label, label_idx
            root_dir (str): Thư mục gốc chứa các ảnh (thường là ./data/raw/embrapa/)
            transform (callable, optional): Transform áp dụng lên ảnh
        """

        self.df = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.root_dir, row["image_path"])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(int(row["label_idx"]), dtype=torch.long)
        return image, label


In [75]:
#just checking
import pandas as pd
df = pd.read_csv('./data/processed/embrapa/train.csv')
print("Số dòng:", len(df))
print(df.head())


Số dòng: 29607
                                          image_path  \
0  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   
1  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   
2  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   
3  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   
4  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...   

                                               label  label_idx  
0  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  
1  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  
2  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  
3  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  
4  Algod├úo (Cotton) - Mancha de Mirotecio (Myrot...          0  


In [76]:
import os

base_dir = "./data/raw/embrapa/train"
class_folders = os.listdir(base_dir)
print("Số class:", len(class_folders))
print("Một số class:", class_folders[:5])

# Lấy thử 1 ảnh từ 1 class
for class_name in class_folders:
    class_path = os.path.join(base_dir, class_name)
    if os.path.isdir(class_path):
        files = os.listdir(class_path)
        print(f"📂 {class_name}: {len(files)} ảnh | VD: {files[:3]}")
        break


Số class: 93
Một số class: ['Milho (Corn) - Ferrugem_Branca (Tropical rust) - Cropped', 'Couve (Cabbage) - Oidio (Powdery mildew) - Cropped', 'Soja (Soybean) - Fito de cobre (Copper Phytotoxicity) - Cropped', 'Coqueiro (Coconut Tree) - Mosca Branca (Whitefly) - Cropped', 'Couve (Cabbage) - Mancha de Alternaria (Alternaria leaf spot) - Cropped']
📂 Milho (Corn) - Ferrugem_Branca (Tropical rust) - Cropped: 568 ảnh | VD: ['fbra707.jpg', 'fbra085.jpg', 'fbra159.jpg']


In [77]:
from torchvision import transforms
from torch.utils.data import DataLoader

# Transform dùng chung cho cả 3 tập
embrapa_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Root thư mục chứa ảnh
image_root = "./data/raw/embrapa/train"

# Tạo dataset
train_dataset = EmbrapaDataset("./data/processed/embrapa/train.csv", image_root, transform=embrapa_transform)
val_dataset = EmbrapaDataset("./data/processed/embrapa/val.csv", image_root, transform=embrapa_transform)
test_dataset = EmbrapaDataset("./data/processed/embrapa/test.csv", image_root, transform=embrapa_transform)

# Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)


training from here


In [78]:
from src.model import PlantXViT

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim

# Load label encoder để biết số lớp
import pickle
with open("./data/processed/embrapa/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)
num_classes = len(label_encoder.classes_)

# Khởi tạo mô hình
model = PlantXViT(num_classes=num_classes).to("cuda")

# Cấu hình optimizer, loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [80]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total


In [81]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total


In [82]:
def train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs, device, save_path="./outputs/models/embrapa/plantxvit_embrapa_best.pth"):
    best_val_loss = float("inf")
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)

        print(f"Epoch [{epoch}/{num_epochs}] "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model.")

    return history


In [83]:
device=torch.device("cuda")
print(device)

cuda


In [84]:

history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=50,
    device=device,
    save_path="./outputs/models/embrapa/plantxvit_embrapa_best.pth"
)


KeyboardInterrupt: 